In [29]:
import redis
import json

r = redis.Redis(host="redis")

# Save and find single users

Let's create two functions to save and get the user:

In [36]:
def save_user(user):
  value = json.dumps(user)
  key = f"users:{user['id']}"
  return r.set(key, value)


def find_user(id):
  key = f"users:{id}"
  value = r.get(key)
  if value is None:
    return False
  return json.loads(value)

Let's create two users and save them:

In [32]:
bob = {'id': 8, 'name': "bob", 'born': 1986, 'country': "DE"}
alice = {'id': 12, 'name': "alice", 'born': 1995, 'country': "DE"}

In [22]:
save_user(bob)

True

In [23]:
save_user(alice)

True

Now, we can try to fetch a user with an invalid id:

In [37]:
find_user(9)

False

Time to fetch them with a valid id:

In [38]:
find_user(8)

{'id': 8, 'name': 'bob', 'born': 1986, 'country': 'DE'}

In [39]:
find_user(12)

{'id': 12, 'name': 'alice', 'born': 1995, 'country': 'DE'}

# Find all users

Currently, there is no way to get a list of all users, let's store an index with all users and a corresponding function to find all users:

In [51]:
def save_user(user):
  # single user
  value = json.dumps(user)
  key = f"users:{user['id']}"
  r.set(key, value)
  
  # add to index
  return r.sadd("user_ids", user['id'])


def find_users():
  # get list of ids
  ids = r.smembers("user_ids")
  
  # build user keys
  keys = [f"users:{id.decode('utf8')}" for id in ids]

  # multi-get all keys at once
  values = r.mget(keys)

  # parse responses into user
  return [json.loads(value) for value in values]

To add users to our index, we need to resave them:

In [49]:
save_user(bob)
save_user(alice)

0

Now, we can find all users:

In [52]:
find_users()

[{'id': 12, 'name': 'alice', 'born': 1995, 'country': 'DE'},
 {'id': 8, 'name': 'bob', 'born': 1986, 'country': 'DE'}]